# UrbanSoundsClassification: AIYard

Model.ipynb

AIYard ekibi

### 1. Kütüphanelerin içe aktarılması

In [4]:
import os
import numpy as np
import tensorflow as tf

### 2. Önişlenmiş verisetlerinin içe aktarılması
Önişleme defterini kullanarak `preproc` klasörü altında oluşturduğumuz `train`, `val` ve `test` klasörlerindeki spektrogramları okuyup demetler halinde kaydederek başlıyoruz.

Google Colab ortamında isek Google Drive'ı mount ettikten sonra `preproc` klasörüne giden yolu tutan `preproc_folder`'ı değiştirmeliyiz.

In [1]:
from google.colab import drive
drive.mount('/content/GDrive/')

Mounted at /content/GDrive/


In [2]:
# DİKKAT: Yolun sonuna "/" karakteri eklemeyi unutma!
# /home/ferit/repos/UrbanSoundsClassification/preproc/
preproc_folder = "/content/GDrive/MyDrive/preproc/"

#### 2.1. `train` klasöründeki spektrogramların okunması

X_train ve Y_train listelerini elde edeceğiz. Her iki listede 6900 tane eleman bulunmalı. 

X_train'e spektrogram verisi olan (64, 64) boyutlu NumPy dizilerini, Y_train'e ise karşılık gelen sınıfın numarasını ekleyeceğiz.

In [5]:
X_train = []
Y_train = []
for path, subdirs, files in os.walk(f"{preproc_folder}train"):
    for name in files:
        file = os.path.join(path, name)
        spectrogram = np.load(file)
        X_train.append(spectrogram)
        Y_train.append(int(file.split('-')[2][0]))

In [6]:
len(X_train) == len(Y_train) == 6900

False

#### 2.2. `val` klasöründeki spektrogramların okunması

X_val ve Y_val listelerini elde edeceğiz. Her iki listede 900 tane eleman bulunmalı.

In [ ]:
X_val = []
Y_val = []
for path, subdirs, files in os.walk(f"{preproc_folder}val"):
    for name in files:
        file = os.path.join(path, name)
        spectrogram = np.load(file)
        X_val.append(spectrogram)
        Y_val.append(int(file.split('-')[2][0]))

In [ ]:
len(X_val) == len(Y_val) == 900

True

#### 2.3. `test` klasöründeki spektrogramların okunması

X_test ve Y_test listelerini elde edeceğiz. Her iki listede 935 tane eleman bulunmalı.

In [ ]:
X_test = []
Y_test = []
for path, subdirs, files in os.walk(f"{preproc_folder}test"):
    for name in files:
        file = os.path.join(path, name)
        spectrogram = np.load(file)
        X_test.append(spectrogram)
        Y_test.append(int(file.split('-')[2][0]))

In [ ]:
len(X_test) == len(Y_test) == 935

True

### 3. Nöral ağın oluşturulması